# Compressible Flow - Inviscid Bump

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from majordome.su2.enums import *
from majordome.su2.groups import *

## Prepare case configuration

In [3]:
conf = SU2Configuration(
    problem = ProblemDefinition(
        solver = SolverType.EULER,
    )
)

conf.compressible_freestream = CompressibleFreeStreamDefinition(
    mach             = 0.5,
    angle_of_attack  = 0.0,
    sideslip_angle   = 0.0,
    pressure         = 101325.0,
    temperature      = 288.0
)

conf.reference_values = ReferenceValues(
    ref_origin_moment_x = 0.25,
    ref_origin_moment_y = 0.00,
    ref_origin_moment_z = 0.00,
    ref_length          = 1.0,
    ref_area            = 1.0
)

conf.boundary_conditions = BoundaryConditions(
    marker_euler  = ["upper_wall", "lower_wall"],
    marker_inlet  = ["inlet", 288.6, 102010.0, 1.0, 0.0, 0.0],
    marker_outlet = ["outlet", 101300.0],
    inlet_type    = InletType.TOTAL_CONDITIONS,
)

conf.surfaces_identification = SurfacesIdentification(
    marker_plotting   = ["lower_wall"],
    marker_monitoring = ["upper_wall", "lower_wall"],
)

conf.common_numerical_parameters = CommonParametersNumerical(
    num_method_grad = NumMethodGrad.GREEN_GAUSS,
    cfl_number      = 50.0,
    cfl_adapt       = YesNoEnum.YES,
    cfl_adapt_param = ParametersCFL(
        factor_down            = 0.1,
        factor_up              = 2.0,
        min_value              = 50.0,
        max_value              = 1.0e+10
    ),
    rk_alpha_coeff = (0.66667, 0.66667, 1.000000),
)

conf.linear_solver_parameters = LinearSolverParameters(
    linear_solver      = LinearSolver.FGMRES,
    linear_solver_prec = Preconditioner.ILU,
    error              = 1.0e-10,
    n_iter             = 20
)

conf.multigrid_parameters = MultigridParameters(
    mg_level             = 3,
    mg_cycle             = MgCycle.W_CYCLE,
    mg_pre_smooth        = [1, 2, 3, 3],
    mg_post_smooth       = [0, 0, 0, 0],
    mg_correction_smooth = [0, 0, 0, 0],
    mg_damp_restriction  = 1.0,
    mg_damp_prolongation = 1.0

)

conf.flow_numerical_method = FlowNumericalMethod(
    conv_num_method_flow = ConvectiveScheme.JST,
    time_discre_flow     = TimeDiscretization.EULER_IMPLICIT,
)

conf.slope_limiter = SlopeLimiter(
    jst_sensor_coeff = (0.5, 0.02)
)

conf.solver_control = SolverControl(
    iter                 = 999999,
    conv_field           = "RMS_DENSITY",
    conv_residual_minval = -10,
    conv_startiter       = 10,
    conv_cauchy_elems    = 100,
    conv_cauchy_eps      = 1.0e-10,
)

conf.io_file_info = IOFileInfo(
    mesh_filename         = "mesh.su2",
    mesh_format           = MeshFormat.SU2,
    mesh_out_filename     = "mesh_out.su2",
    solution_filename     = "solution_flow",
    solution_adj_filename = "solution_adj",
    tabular_format        = TabularFormat.CSV,
    conv_filename         = "history.dat",
    restart_filename      = "restart_flow",
    restart_adj_filename  = "restart_adj",
    volume_filename       = "flow",
    volume_adj_filename   = "adjoint",
    grad_objfunc_filename = "of_grad.dat",
    surface_filename      = "surface_flow",
    surface_adj_filename  = "surface_adjoint",
)

conf.to_file("model.cfg", force=True)

## Run simulation